<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/10_shapefiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [11]:
import geemap
import ee
import os
import geemap.colormaps as cm
from geemap.datasets import DATA

In [2]:
ee.Initialize()

In [3]:
Map = geemap.Map(center=(-16,-56),zoom=4)
Map

Map(center=[-16, -56], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=…

In [21]:
#you can put any .shp here to extract the data
COUNTRY_shp = '/home/joseacm/projetos/DOCTORADE/PauloT/FOREST_COVER/SHP_BR/ne_10m_admin_0_countries.shp'
COUNTRY = geemap.shp_to_ee(COUNTRY_shp)
Map.addLayer(COUNTRY, {}, 'COUNTRY BORDER')

EEException: Request payload size exceeds the limit: 4194304 bytes.

In [5]:
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

In [14]:
Landcover = ee.Image("COPERNICUS/Landcover/100m/Proba-V-C3/Global/2019").select('discrete_classification')

clipped_Landcover = Landcover.clip(COUNTRY)

Map.addLayer(clipped_Landcover, {}, "Land Cover")

In [16]:

Forest = ee.Image("JAXA/ALOS/PALSAR/YEARLY/FNF/2017").select('fnf')

clipped_forest = Forest.clip(COUNTRY)

Map.addLayer(clipped_forest, {}, "forest Cover")

In [17]:
collection = ee.ImageCollection('IDAHO_EPSCOR/TERRACLIMATE')\
    .filterDate('2015-01-01', '2020-01-01')\
    .select('pr')\
  
median = collection.median().clip(COUNTRY)
palette = cm.palettes.cool
vis = {
  'min': 0,
  'max': 300.0,
  'palette': palette
}
Map.addLayer(median, vis, 'median')

Idahodata = collection.toBands().clip(COUNTRY)

These codes will download the data to your local computer. I set then to Raw NBConvert, if you want to download, change the tipe for code

corine_stats = os.path.join(out_dir, 'Landcovercorine2019.csv')  

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
#statistics_type can be either 'SUM' or 'PERCENTAGE'
#denominator can be used to convert square meters to other areal units, such as square kilimeters
geemap.zonal_statistics_by_group(clipped_Landcover, COUNTRY, corine_stats, statistics_type='PERCENTAGE', denominator=1000000, decimal_places=2)

out_landsat_stats = os.path.join(out_dir, 'ndvi.csv')  
geemap.zonal_statistics(Idahodata, COUNTRY, out_landsat_stats, statistics_type='MEAN', scale=1000)
